# HuggingFace: Summary of the Task:: MLM [link](https://huggingface.co/transformers/task_summary.html?highlight=fill%20mask#masked-language-modeling)

In [131]:
from transformers import pipeline
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast , AutoTokenizer,RobertaTokenizerFast, RobertaTokenizer
import torch
import numpy as np

In [239]:
tokenizer = RobertaTokenizer.from_pretrained("./all-data-bytebpe-20000", max_len=510) # since there are 2 special tokens?
tokenizer.__class__.__name__

'RobertaTokenizer'

In [10]:
model = AutoModelWithLMHead.from_pretrained("./Roberta_DonutTest/checkpoint-200000")

In [11]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(20000, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [14]:
model.num_parameters()

102012704

In [12]:
nlp = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [13]:
nlp(f"HuggingFace is creating a {nlp.tokenizer.mask_token} that the community uses to solve NLP tasks.")

[{'sequence': '<s>HuggingFace is creating a lot that the community uses to solve NLP tasks.</s>',
  'score': 0.27496138215065,
  'token': 15888,
  'token_str': 'Ġlot'},
 {'sequence': '<s>HuggingFace is creating age that the community uses to solve NLP tasks.</s>',
  'score': 0.14158375561237335,
  'token': 3789,
  'token_str': 'ge'},
 {'sequence': '<s>HuggingFace is creating away that the community uses to solve NLP tasks.</s>',
  'score': 0.042050398886203766,
  'token': 6149,
  'token_str': 'way'},
 {'sequence': '<s>HuggingFace is creating a way that the community uses to solve NLP tasks.</s>',
  'score': 0.023051628842949867,
  'token': 13919,
  'token_str': 'Ġway'},
 {'sequence': '<s>HuggingFace is creating a relative that the community uses to solve NLP tasks.</s>',
  'score': 0.02065601758658886,
  'token': 17586,
  'token_str': 'Ġrelative'}]

In [51]:
nlp(f"วันนี้เป็นวันที่{nlp.tokenizer.mask_token}")

[{'sequence': '<s>วันนี้เป็นวันที่ 2</s>',
  'score': 0.13160008192062378,
  'token': 403,
  'token_str': 'Ġ2'},
 {'sequence': '<s>วันนี้เป็นวันที่ 1</s>',
  'score': 0.0837036520242691,
  'token': 395,
  'token_str': 'Ġ1'},
 {'sequence': '<s>วันนี้เป็นวันที่ 3</s>',
  'score': 0.0767679437994957,
  'token': 485,
  'token_str': 'Ġ3'},
 {'sequence': '<s>วันนี้เป็นวันที่ท</s>',
  'score': 0.05507468059659004,
  'token': 288,
  'token_str': 'à¸Ĺ'},
 {'sequence': '<s>วันนี้เป็นวันที่ส</s>',
  'score': 0.04600020870566368,
  'token': 294,
  'token_str': 'à¸ª'}]

## HuggingFace Trainer [link](https://github.com/huggingface/transformers/blob/master/src/transformers/trainer.py#L375)

## DataCollator [link](https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py#L69)

1. A torch.utils.data.dataloader.Dataloader is created from a Dataset (where the examples are tokenized with `tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))` and appended special tokens `<s>` and `</s>` through `tokenizer.build_inputs_with_special_tokens(tokenized_text[i : i + self.block_size])` [this is created where `block_size` is `self.block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)`] ) referenced [here](https://github.com/huggingface/transformers/blob/9022ef021a56db975d25c7108cbd19d0dd399174/src/transformers/trainer.py#L224).  
This will return a "List" of ids. 
2. [_tensorize_batch()](https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py#L90) will attempt to convert from ```examples: List[torch.Tensor]) -> torch.Tensor:``` and also apply [self.tokenizer._pad_token()](https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py#L101) along the way for examples with irregular length
3. ```if self.mlm: ``` , mask the tokens with [mask_tokens()](https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py#L103) to Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.

In [57]:
sequence = f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help {tokenizer.mask_token} our carbon footprint."

In [63]:
_input = tokenizer.encode(sequence, return_tensors="pt")
print(list(map(lambda x: tokenizer.decode([x]) , _input[0])))
_input

['<s>', 'D', 'ist', 'il', 'led', ' model', 's', ' are', ' sm', 'all', 'er', ' than', ' the', ' model', 's', ' they', ' m', 'im', 'ic', '.', ' ', 'U', 'sing', ' the', 'm', ' inst', 'e', 'ad', ' of', ' the', ' l', 'ar', 'ge', ' vers', 'ions', ' w', 'ould', ' hel', 'p', '<mask>', ' our', ' car', 'b', 'on', ' f', 'oot', 'pr', 'int', '.', '</s>']


tensor([[    0,    40,  2744,   789,  9774, 17624,    87,  6344,  2352,  1848,
           522, 13701,  1207, 17624,    87, 19898,   584,  1164,   721,    18,
           225,    57, 19100,  1207,    81, 12140,    73,   977,  1572,  1207,
           676,   633,  3789, 12847, 10865,   731, 11955, 15051,    84,     4,
         14855,  5963,    70,   524,   655,  8489,  3220,  3449,    18,     2]])

In [66]:
labels = _input.clone()
# We sample a few tokens in each sequence for masked-LM training (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
probability_matrix = torch.full(labels.shape, 0.15)
probability_matrix

tensor([[0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500]])

In [72]:
# If there is special tokens, fill it with 0.0
special_tokens_mask = [
    tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
]
print(special_tokens_mask)
probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
probability_matrix

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]


tensor([[0.0000, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500, 0.1500,
         0.1500, 0.1500, 0.1500, 0.1500, 0.0000]])

In [71]:
# IF there is padding mask, fill it with 0 too
padding_mask = labels.eq(tokenizer.pad_token_id)
probability_matrix.masked_fill_(padding_mask, value=0.0)
padding_mask

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False]])

In [75]:
# Randomly select which one to mask based on the probabilities given ealier, and if the mask isn't chosen, fill in -100 for the value
masked_indices = torch.bernoulli(probability_matrix).bool()
print(masked_indices)
labels[~masked_indices] = -100  # We only compute loss on masked 
labels

tensor([[False, False, False,  True,  True, False, False, False,  True,  True,
         False, False, False, False,  True, False, False, False,  True, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True, False, False, False,  True, False,
         False,  True, False, False, False, False, False, False,  True, False]])


tensor([[ -100,  -100,  -100,   789,  9774,  -100,  -100,  -100,  2352,  1848,
          -100,  -100,  -100,  -100,    87,  -100,  -100,  -100,   721,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100, 10865,  -100,  -100,  -100,    84,  -100,
          -100,  5963,  -100,  -100,  -100,  -100,  -100,  -100,    18,  -100]])

In [77]:
# 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
_input[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

# 10% of the time, we replace masked input tokens with random word
indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
random_words = torch.randint(len(tokenizer), labels.shape, dtype=torch.long)
_input[indices_random] = random_words[indices_random]

# The rest of the time (10% of the time) we keep the masked input tokens unchanged
print(_input, labels)

tensor([[    0,    40,  2744,     4,     4, 17624,    87,  6344,     4,     4,
           522, 13701,  1207, 17624,     4, 19898,   584,  1164,     4,    18,
           225,    57, 19100,  1207,    81, 12140,    73,   977,  1572,  1207,
           676,   633,  3789, 12847,     4,   731, 11955, 15051,    84,     4,
         14855,     4,    70,   524,   655,  8489,  3220,  3449,    18,     2]]) tensor([[ -100,  -100,  -100,   789,  9774,  -100,  -100,  -100,  2352,  1848,
          -100,  -100,  -100,  -100,    87,  -100,  -100,  -100,   721,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100, 10865,  -100,  -100,  -100,    84,  -100,
          -100,  5963,  -100,  -100,  -100,  -100,  -100,  -100,    18,  -100]])


In [78]:
print(list(map(lambda x: tokenizer.decode([x]) , _input[0])))

['<s>', 'D', 'ist', '<mask>', '<mask>', ' model', 's', ' are', '<mask>', '<mask>', 'er', ' than', ' the', ' model', '<mask>', ' they', ' m', 'im', '<mask>', '.', ' ', 'U', 'sing', ' the', 'm', ' inst', 'e', 'ad', ' of', ' the', ' l', 'ar', 'ge', ' vers', '<mask>', ' w', 'ould', ' hel', 'p', '<mask>', ' our', '<mask>', 'b', 'on', ' f', 'oot', 'pr', 'int', '.', '</s>']


# Now I'm going to define `_mask_input()` 
 0.15 Probability: With 100% replace with mask, and 0% unchanged

In [231]:
def _mask_input(sequence, tokenizer, mlm_probability=0.15, truncation=True):
    # Tokenize the sequence with special tokens inserted
    _input = tokenizer.encode(sequence, return_tensors="pt", truncation=truncation)
    
    labels = _input.clone()
    # We sample a few tokens in each sequence for masked-LM training (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
    probability_matrix = torch.full(labels.shape, mlm_probability)
    
    # If there is special tokens, fill it with 0.0
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    
    # IF there is padding mask, fill it with 0 too
    padding_mask = labels.eq(tokenizer.pad_token_id)
    probability_matrix.masked_fill_(padding_mask, value=0.0)
    
    # Randomly select which one to mask based on the probabilities given ealier, and if the mask isn't chosen, fill in -100 for the value
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100  # We only compute loss on masked 
    
    # 100% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
    indices_replaced = masked_indices
    _input[indices_replaced] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)
    
    return _input, labels

In [90]:
tokenized_input, labels = _mask_input(f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help our carbon footprint.", tokenizer)
print(tokenized_input)
print(list(map(lambda x: tokenizer.decode([x]) , tokenized_input[0])))
print(labels)
print(list(map(lambda x: tokenizer.decode([x]) if x != -100 else '-', labels[0])))

tensor([[    0,    40,  2744,   789,  9774, 17624,    87,     4,  2352,     4,
             4, 13701,  1207, 17624,    87, 19898,   584,  1164,   721,    18,
           225,     4, 19100,  1207,    81, 12140,    73,   977,  1572,  1207,
           676,   633,  3789, 12847, 10865,   731, 11955,     4,    84, 14855,
          5963,    70,   524,     4,  8489,  3220,  3449,     4,     2]])
['<s>', 'D', 'ist', 'il', 'led', ' model', 's', '<mask>', ' sm', '<mask>', '<mask>', ' than', ' the', ' model', 's', ' they', ' m', 'im', 'ic', '.', ' ', '<mask>', 'sing', ' the', 'm', ' inst', 'e', 'ad', ' of', ' the', ' l', 'ar', 'ge', ' vers', 'ions', ' w', 'ould', '<mask>', 'p', ' our', ' car', 'b', 'on', '<mask>', 'oot', 'pr', 'int', '<mask>', '</s>']
tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  6344,  -100,  1848,
           522,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,    57,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100

# Now I'm going to define `_predict_masks()`

In [272]:
def _predict_masks(_input, labels, tokenizer, k=5, verbose=True, verbose_length=6):
    mask_token_index = torch.where(_input == tokenizer.mask_token_id)[1]
#     print(mask_token_index)
    token_logits = model(_input)[0]
#     print(token_logits)
    mask_token_logits = token_logits[0, mask_token_index, :]
#     print(mask_token_logits)
    top_k_tokens = torch.topk(mask_token_logits, k, dim=1).indices.numpy()
    _labels = labels[0, mask_token_index].numpy()
    _labels = np.vstack(_labels) #vertical stack to be able to compare to top_5_tokens
#     print(top_5_tokens)
#     print(_labels)
    
    for index, mask_index in enumerate(mask_token_index):
        input_verbose_sequence = _input[0, mask_index-verbose_length:mask_index+verbose_length].clone()
        labeled_verbose_sequence = input_verbose_sequence.clone()
        labeled_verbose_sequence[verbose_length] = labels[0, mask_index]
        print("Masked Input: ")
#         print(f"\t{list(map(lambda x: tokenizer.decode([x]) , input_verbose_sequence))}")
        print(f"\t{tokenizer.decode(input_verbose_sequence)}")
        print("Labeled Input: ")
#         print(f"\t{list(map(lambda x: tokenizer.decode([x]) , labeled_verbose_sequence))}")
        print(f"\t{tokenizer.decode(labeled_verbose_sequence)}")
        print("Predictions: ")
        for prediction in top_k_tokens[index]:
            prediction_verbose_sequence = input_verbose_sequence.clone()
            prediction_verbose_sequence[verbose_length] = prediction
#             print(f"\t{list(map(lambda x: tokenizer.decode([x]) , prediction_verbose_sequence))}")
            print(f"\t{tokenizer.decode(prediction_verbose_sequence)}")
        print("")
    
    correct_labels = np.any(top_k_tokens == _labels, axis=1)
    percent_correct = np.sum(correct_labels)/correct_labels.size
    print(f"Total Input Tokens: {_input.squeeze().shape}")
    print(f"Total Correct for Top {k}: {np.sum(correct_labels)}")
    print(f"Total Mask: {correct_labels.size}")
    print(f"Percent Correct: {percent_correct*100:.2f}%")
    return percent_correct

In [221]:
_predict_masks(tokenized_input, labels, tokenizer , k=5, verbose=True, verbose_length=6)

Masked Input: 
	['D', 'ist', 'il', 'led', ' model', 's', '<mask>', ' sm', '<mask>', '<mask>', ' than', ' the']
Labeled Input: 
	['D', 'ist', 'il', 'led', ' model', 's', ' are', ' sm', '<mask>', '<mask>', ' than', ' the']
Predictions: 
	['D', 'ist', 'il', 'led', ' model', 's', ' they', ' sm', '<mask>', '<mask>', ' than', ' the']
	['D', 'ist', 'il', 'led', ' model', 's', ' we', ' sm', '<mask>', '<mask>', ' than', ' the']
	['D', 'ist', 'il', 'led', ' model', 's', ' and', ' sm', '<mask>', '<mask>', ' than', ' the']
	['D', 'ist', 'il', 'led', ' model', 's', ' that', ' sm', '<mask>', '<mask>', ' than', ' the']
	['D', 'ist', 'il', 'led', ' model', 's', ' you', ' sm', '<mask>', '<mask>', ' than', ' the']

Masked Input: 
	['il', 'led', ' model', 's', '<mask>', ' sm', '<mask>', '<mask>', ' than', ' the', ' model', 's']
Labeled Input: 
	['il', 'led', ' model', 's', '<mask>', ' sm', 'all', '<mask>', ' than', ' the', ' model', 's']
Predictions: 
	['il', 'led', ' model', 's', '<mask>', ' sm', 'ooth'

0.5714285714285714

# Trying Mask Filling on New Unseen Pantip Samples

### Test Tokenize on Pantip Sample ใครเคยมีแฟนที่กินอาหารไม่ถูกปากกันแล้วรู้สึกเสียความสุขไปอย่างนึงบ้างมั้ยครับ
https://pantip.com/topic/40006922

In [276]:
text = "ใครเคยมีแฟนที่กินอาหารไม่ถูกปากกันแล้วรู้สึกเสียความสุขไปอย่างนึงบ้างมั้ยครับ  ก่อนอื่นผมต้องบอกก่อนเลยว่าคนเราจะเลือกกินอาหารแบบไหนชอบแบบไหนเป็นเรื่องของความชอบส่วนตัวนะครับทุกคนมีสิทธิในการเลือกของที่ชอบและไม่ชอบอยู่แล้ว แต่ผมรู้สึกว่าตอนนี้ผมกำลังประสบปัญหาที่ดูเหมือนจะเล็กแต่กลายเป็นว่ามันค่อนข้างใหญ่ ผมคบกับแฟนมา6ปีแล้วครับ ผมเป็นคนชอบกินอาหารญี่ปุ่นและปลาดิบแต่แฟนผมไม่กินปลาดิบเลย ผมอยากกินบุฟเฟ่เนื้อแต่แฟนผมก็ไม่กินเนื้อ เราเลยไม่ได้เข้าทานร้านบุฟเฟ่เนื้อและบุฟเฟ่อาหารญี่ปุ่นกันเพราะรู้สึกลัวแฟนผมทานไม่คุ้ม และเรื่องใหญ่เลยคือผมเป็นคนชอบทานอาหารรสจัดและรสเผ็ดมาก แต่แฟนผมทานเผ็ดไม่ได้เลยเวลาเราไปกินส้มตำกันก็จะสั่ง ส้มตำไม่ใส่พริก ต้มแซ่บไม่ใส่พริก ลาบไม่ใส่พริก ร้านกับข้าวอื่นๆก็เช่นกันแฟนผมจะไม่ชอบกินผักไม่ค่อยสั่งกับข้าวที่เป็นผักแล้วผมชอบผักบุ้งทอดกรอบ เห็ดหอมสดทอดมาก แต่ก็ไม่ได้สั่งเพราะว่าเธอไม่กินถึงเค้าจะบอกให้สั่งเลยๆก็เถอะแต่ผมก็ยังเกรงใจเธออยู่ดีอ่ะครับ ผมรู้สึกกินอาหารไม่มีความสุขเลยชีวิตผมขาดรสเผ็ดไปเหมือนจะขาดใจเหมือนมันทำให้ขาดความสุขไปอย่างนึงเลยอ่ะครับ ยิ่งถ้าเราแต่งงานกันแล้วผมก็อาจจะต้องมีปัญหาเรื่องนี้มากขึ้น พอผมเห็นคู่ที่ชอบทานอาหารเหมือนๆกันเห็นเค้ากินอาหารกันอย่างมีความสุขแล้วผมรู้สึกอิจฉามากๆเลย มีใครเคยมีปัญหาแบบผมมั้ยครับแล้วจะแก้ปัญหานี้ยังไงดีครับ"
tokenized_input, labels = _mask_input(text, tokenizer, mlm_probability=0.1)
_predict_masks(tokenized_input, labels, tokenizer , k=5, verbose=True, verbose_length=6)

Masked Input: 
	นอาหารไม่ถูกป<mask><mask>นแล้วรู้
Labeled Input: 
	นอาหารไม่ถูกปากก<mask>นแล้วรู้
Predictions: 
	นอาหารไม่ถูกปนก<mask>นแล้วรู้
	นอาหารไม่ถูกปนๆก<mask>นแล้วรู้
	นอาหารไม่ถูกปองก<mask>นแล้วรู้
	นอาหารไม่ถูกปากก<mask>นแล้วรู้
	นอาหารไม่ถูกปอดก<mask>นแล้วรู้

Masked Input: 
	ไม่ถูกป<mask><mask>นแล้วรู้ส
Labeled Input: 
	ไม่ถูกป<mask>ันแล้วรู้ส
Predictions: 
	ไม่ถูกป<mask>ันแล้วรู้ส
	ไม่ถูกป<mask>็นแล้วรู้ส
	ไม่ถูกป<mask>ั้นแล้วรู้ส
	ไม่ถูกป<mask>ินแล้วรู้ส
	ไม่ถูกป<mask>นจนแล้วรู้ส

Masked Input: 
	ึงบ้างมั้ยคร<mask>บ  ก่อนอ
Labeled Input: 
	ึงบ้างมั้ยครับ  ก่อนอ
Predictions: 
	ึงบ้างมั้ยครับ  ก่อนอ
	ึงบ้างมั้ยคร้บ  ก่อนอ
	ึงบ้างมั้ยครั้บ  ก่อนอ
	ึงบ้างมั้ยครีบ  ก่อนอ
	ึงบ้างมั้ยครั๊บ  ก่อนอ

Masked Input: 
	<mask>บ  ก่อนอ<mask>นผมต้องบอกก่
Labeled Input: 
	<mask>บ  ก่อนอื่นผมต้องบอกก่
Predictions: 
	<mask>บ  ก่อนอื่นผมต้องบอกก่
	<mask>บ  ก่อนอืนผมต้องบอกก่
	<mask>บ  ก่อนอั้นผมต้องบอกก่
	<mask>บ  ก่อนอิ่นผมต้องบอกก่
	<mask>บ  ก่อนอื้นผมต้องบอกก่

Masked Input: 
	่อนเลยว่าคน

0.9649122807017544

### Test Tokenize on Pantip Sample อาการแบบนี้คือไรกัน?
https://pantip.com/topic/40009518

In [274]:
text = "อาการแบบนี้คือไรกัน?  เขาคุยกับเรามา 5-6 เดือน เราตามจีบเขานะคะ ก็คุยกันมา ในระยะเวลาเขาบอกว่า ถ้าเราลด นน เพื่อเขาได้ เขาจะยอมเป็นแฟนเรา ตรรกะโง่มากนะคะ แต่ถามว่าทำมั้ย ทำค่ะ พอไปรู้ว่าเขาคุยกับเพื่อน เพื่อนเขาถามว่า รู้สึกยังไงกับเรา เขาตอบเพื่อนว่า เขาว่าเขาควรอยู่คนเดียว ยังไม่พร้อมจะรักใคร จนตอนนี้เราเริ่มรู้สึกว่า ทำไมเราต้องทำขนาดนั้น ถ้าเขาจะรัก รักที่เป็นตัวเราไม่ได้หรอ หลังๆเลยเริ่มสนใจเขาน้อยลง แต่ยังคุยกันเหมือนเดิม เราลองแกล้งเงียบไป ไม่ทักไปครึ่งวัน ปกติเราจะมีการมอนิ่งกันตอนเช้าค่ะ พอเราไม่ทักไป เขาทำงานเสร็จ ถึงเวลาพักของเขา เขาก็ทักมาว่า กินข้าวกัน เราก็ยิ่ง งง ก็คิดว่า เขาอาจจะชินหับการคุยกับเราทุกวันเฉยๆ นี่เลยไม่ได้สนใจในส่วนนั้น เราก็ตอบตามปกติ จนเมื่อคืนมีคนมาทักเราจีบเรา จะไปส่งเราที่บ้าน เราก็เลยเล่าให้เขาฟังว่า ให้ไลน์ไป ให้เขาไปส่งอยู่แต่ไมไ่ด้นั่งรถคันเดียวกัน เราก็ขับของเรา คนที่มาจีบเราเขาก็ขับคันของเขาแค่มาส่งเฉยๆ พอเช้ามาเขาทักมามอนิ่ง ก็ถามเราเรื่องเมื่อคืน เราทำงานที่กลับดึกมากๆไม่ได้ทักไปบอกเขาไว้ว่า ถึงบ้านแล้วนะ เงียบไปทั้งคืนเลย เขาก็ถามเรื่องเมื่อคืนว่า หนุ่มไปส่งที่บ้านเป็นไงบ้าง ถามแต่เรื่องของผู้ชายที่มาจีบเราทั้งวัน จนเราเปลี่ยนเรื่องก็ยังกลับมาถามอีกรอบ ไออาการแบบนี้คืออะไรคะ ? ไหนเขาบอกอยากอยู่คนเดียว แต่พอเรามีคนเข้ามา ทำไมเขาถึงมีอาการแบบนี้ มาถามแบบนี้ซ้ำๆ คืออะไรกัน เราไม่อยากคิดอะไรไปเอง ใครพอจะตอบได้บ้างคะ ว่า ไอแบบนี้มันคืออะไร รู้สึกอะไรอยู่"
tokenized_input, labels = _mask_input(text, tokenizer, mlm_probability=0.1)
_predict_masks(tokenized_input, labels, tokenizer , k=5, verbose=True, verbose_length=6)

Masked Input: 
	ี้คือไรกั<mask>?  เขาคุ<mask>
Labeled Input: 
	ี้คือไรกัน?  เขาคุ<mask>
Predictions: 
	ี้คือไรกัน?  เขาคุ<mask>
	ี้คือไรกับเรา?  เขาคุ<mask>
	ี้คือไรกันคะ?  เขาคุ<mask>
	ี้คือไรกันหรอ?  เขาคุ<mask>
	ี้คือไรกันไหม?  เขาคุ<mask>

Masked Input: 
	ั<mask>?  เขาคุ<mask>ับเรามา<mask>-
Labeled Input: 
	ั<mask>?  เขาคุยกับเรามา<mask>-
Predictions: 
	ั<mask>?  เขาคุยกับเรามา<mask>-
	ั<mask>?  เขาคุยอะไรกับเรามา<mask>-
	ั<mask>?  เขาคุยกลับเรามา<mask>-
	ั<mask>?  เขาคุบกับเรามา<mask>-
	ั<mask>?  เขาคุยครับเรามา<mask>-

Masked Input: 
	 เขาคุ<mask>ับเรามา<mask>-6 เดื<mask>
Labeled Input: 
	 เขาคุ<mask>ับเรามา 5-6 เดื<mask>
Predictions: 
	 เขาคุ<mask>ับเรามา 5-6 เดื<mask>
	 เขาคุ<mask>ับเรามา 3-6 เดื<mask>
	 เขาคุ<mask>ับเรามา 4-6 เดื<mask>
	 เขาคุ<mask>ับเรามา 2-6 เดื<mask>
	 เขาคุ<mask>ับเรามา5-6 เดื<mask>

Masked Input: 
	ามา<mask>-6 เดื<mask><mask>าตามจีบเข
Labeled Input: 
	ามา<mask>-6 เดือน<mask>าตามจีบเข
Predictions: 
	ามา<mask>-6 เดือน<mask>าตามจีบเข
	ามา<mask>-6 เดือนเรา<ma

0.9649122807017544

### Test Tokenize on Pantip Sample ร่างจำแลง (Literature)
https://pantip.com/topic/40039192

In [279]:
text = """ก้าวแรกของผมที่สัมผัสฝ่าเท้าลงภายในสถานเริงรมย์แห่งนั้น เสียงต่างๆรวมถึงภาพเคลื่อนไหวประดังประเดเข้าในโสตอย่างพร้อมเพรียง สร้างความอึกทึกมึนตึงไปหมด
แสงสีบนเพดานที่กระพริบกวัดแกว่งไปมาจากการควบคุมด้วยเทคโนโลยี สลับลำแสงท่ามกลางความมืดจนตาลายจำแนกวัตถุไม่ออก ยิ่งสำหรับคนที่ไม่เคยชินอย่างผมที่เพิ่งย่างกรายเข้ามาเป็นหนแรก
เสียงเพลงก็กระหึ่มกระแทกกระทั้นเร้าใจ กระตุ้นให้ทุกอณูส่วนของร่างกายอยากขยับเด้าดิ้นวาดลวดลายจนทรุดกองฮวบกับพื้น ดุจดั่งที่หลายคนบนพื้นเต้นกำลังทำอยู่ หรือมิเช่นนั้นก็หนวกหูรำคาญไปเลยอย่างที่ผมกำลังรู้สึก
ชัยนาทเห็นผมแต่ไกลได้อย่างไรไม่รู้ เขาคงคุ้นเคยกับห้องมืดสลัวที่มีแสงไฟเปลี่ยนสีตวัดกวัดไกวไปมาจนปวดตา จึงเห็นผมได้ไม่ยากเย็น"""
tokenized_input, labels = _mask_input(text, tokenizer, mlm_probability=0.1)
_predict_masks(tokenized_input, labels, tokenizer , k=5, verbose=True, verbose_length=6)

Masked Input: 
	รมย์แห่งนั้<mask> เสียงต่<mask>
Labeled Input: 
	รมย์แห่งนั้น เสียงต่<mask>
Predictions: 
	รมย์แห่งนั้น เสียงต่<mask>
	รมย์แห่งนั้นเอง เสียงต่<mask>
	รมย์แห่งนั้นเลย เสียงต่<mask>
	รมย์แห่งนั้นๆ เสียงต่<mask>
	รมย์แห่งนั้นมา เสียงต่<mask>

Masked Input: 
	ั้<mask> เสียงต่<mask>รวมถึงภาพเคลื่
Labeled Input: 
	ั้<mask> เสียงต่างๆรวมถึงภาพเคลื่
Predictions: 
	ั้<mask> เสียงต่างรวมถึงภาพเคลื่
	ั้<mask> เสียงต่างๆรวมถึงภาพเคลื่
	ั้<mask> เสียงต่างออกไปรวมถึงภาพเคลื่
	ั้<mask> เสียงต่างหากรวมถึงภาพเคลื่
	ั้<mask> เสียงต่างประเทศรวมถึงภาพเคลื่

Masked Input: 
	ื่อนไหวประดังประเดเข<mask>าในโสตอย่
Labeled Input: 
	ื่อนไหวประดังประเดเข้าในโสตอย่
Predictions: 
	ื่อนไหวประดังประเดเข้าในโสตอย่
	ื่อนไหวประดังประเดเข่าในโสตอย่
	ื่อนไหวประดังประเดเขลาในโสตอย่
	ื่อนไหวประดังประเดเขมาในโสตอย่
	ื่อนไหวประดังประเดเขตาในโสตอย่

Masked Input: 
	าในโสตอย่างพร<mask><mask>รียง สร
Labeled Input: 
	าในโสตอย่างพร้<mask>รียง สร
Predictions: 
	าในโสตอย่างพร้<mask>รียง สร
	าในโสตอย่างพรุ<mask>รียง สร

0.8571428571428571

### Test Tokenize on Own Sample

In [547]:
text = "สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว นี่คือการเว้นวรรคสองทีครับ จะได้ออกเป็นสอง"
tokenized_input, labels = _mask_input(text, tokenizer, mlm_probability=0.15)
_predict_masks(tokenized_input, labels, tokenizer , k=5, verbose=True, verbose_length=6)

Masked Input: 
	ัสดีครับ<mask>ื่อไนท์ ตอนน
Labeled Input: 
	ัสดีครับ ผมชื่อไนท์ ตอนน
Predictions: 
	ัสดีครับ ผมชื่อไนท์ ตอนน
	ัสดีครับ เราชื่อไนท์ ตอนน
	ัสดีครับ เพื่อไนท์ ตอนน
	ัสดีครับ ชื่อไนท์ ตอนน
	ัสดีครับ เมื่อไนท์ ตอนน

Masked Input: 
	้องไปโรงเรียนแล้<mask> น<mask>คือการ
Labeled Input: 
	้องไปโรงเรียนแล้ว น<mask>คือการ
Predictions: 
	้องไปโรงเรียนแล้ว น<mask>คือการ
	้องไปโรงเรียนแล้วนะ น<mask>คือการ
	้องไปโรงเรียนแล้วผม น<mask>คือการ
	้องไปโรงเรียนแล้วว น<mask>คือการ
	้องไปโรงเรียนแล้ววว น<mask>คือการ

Masked Input: 
	โรงเรียนแล้<mask> น<mask>คือการ<mask>้
Labeled Input: 
	โรงเรียนแล้<mask> นี่คือการ<mask>้
Predictions: 
	โรงเรียนแล้<mask> นี่คือการ<mask>้
	โรงเรียนแล้<mask> นี้คือการ<mask>้
	โรงเรียนแล้<mask> นีคือการ<mask>้
	โรงเรียนแล้<mask> นิคือการ<mask>้
	โรงเรียนแล้<mask> น็คือการ<mask>้

Masked Input: 
	<mask> น<mask>คือการ<mask>้นวรรคสองที
Labeled Input: 
	<mask> น<mask>คือการเว้นวรรคสองที
Predictions: 
	<mask> น<mask>คือการเว้นวรรคสองที
	<mask> น<mask>คือการลดต้นวรรคส

0.8571428571428571

Or trying the Vanilla way to generate

In [15]:
sequence = f"Distilled models are smaller than the models they mimic. Using them instead of the large versions would help {tokenizer.mask_token} our carbon footprint."

In [19]:
_input = tokenizer.encode(sequence, return_tensors="pt")
_input

tensor([[    0,    40,  2744,   789,  9774, 17624,    87,  6344,  2352,  1848,
           522, 13701,  1207, 17624,    87, 19898,   584,  1164,   721,    18,
           225,    57, 19100,  1207,    81, 12140,    73,   977,  1572,  1207,
           676,   633,  3789, 12847, 10865,   731, 11955, 15051,    84,     4,
         14855,  5963,    70,   524,   655,  8489,  3220,  3449,    18,     2]])

In [53]:
tokenizer.encode_plus(sequence)

{'input_ids': [0, 40, 2744, 789, 9774, 17624, 87, 6344, 2352, 1848, 522, 13701, 1207, 17624, 87, 19898, 584, 1164, 721, 18, 225, 57, 19100, 1207, 81, 12140, 73, 977, 1572, 1207, 676, 633, 3789, 12847, 10865, 731, 11955, 15051, 84, 4, 14855, 5963, 70, 524, 655, 8489, 3220, 3449, 18, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
mask_token_index = torch.where(_input == tokenizer.mask_token_id)[1]
mask_token_index

tensor([39])

In [22]:
token_logits = model(_input)[0]
token_logits

tensor([[[ -6.8569, -11.4544,  -6.2892,  ...,  -3.3654,  -1.7394,  -3.8599],
         [ -6.7165,  -7.8614, -10.1110,  ...,  -0.2116,  -1.2954,  -8.0255],
         [ -6.4925,  -9.5634,  -9.8982,  ...,  -3.4279,  -1.0313,  -6.8084],
         ...,
         [ -6.2978,  -6.5940,  -9.4618,  ...,   0.5986,  -2.2195,  -8.1321],
         [ -6.3295,  -9.3629,  -9.6262,  ...,   0.0172,  -8.5017,  -5.1815],
         [ -6.8569, -11.4543,  -6.2891,  ...,  -3.3655,  -1.7393,  -3.8597]]],
       grad_fn=<AddBackward0>)

In [23]:
mask_token_logits = token_logits[0, mask_token_index, :]
mask_token_logits

tensor([[ -6.0691,  -7.5146, -11.4188,  ...,   3.1234,  -1.2106,  -6.2130]],
       grad_fn=<IndexBackward>)

In [24]:
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
top_5_tokens

[1228, 1187, 1796, 18, 12549]

In [25]:
for token in top_5_tokens:
     print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

Distilled models are smaller than the models they mimic. Using them instead of the large versions would help ed our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help  in our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help  to our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help . our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help  her our carbon footprint.


# Testing Text-Generation

In [26]:
text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [27]:
text_gen("As far as I am concerned, I will", max_length=50, do_sample=False)

NotImplementedError: Generation is currently not supported for RobertaForMaskedLM. Please select a model from ['XLNetLMHeadModel', 'TransfoXLLMHeadModel', 'ReformerModelWithLMHead', 'GPT2LMHeadModel', 'OpenAIGPTLMHeadModel', 'CTRLLMHeadModel', 'TFXLNetLMHeadModel', 'TFTransfoXLLMHeadModel', 'TFGPT2LMHeadModel', 'TFOpenAIGPTLMHeadModel', 'TFCTRLLMHeadModel'] for generation.

In [281]:
prompt = "สวัสดีครับ"
_inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
print(_inputs)
[tokenizer.decode([tok]) for tok in _inputs[0]]

tensor([[1783,  275,  566,  278,  333,  275,  282]])


['สว', 'ั', 'สด', 'ี', 'คร', 'ั', 'บ']

In [488]:
# def _text_generation(prompt, max_length=250)
prompt = "สวัสดีครับ"

def _add_mask_to_last(prompt, tokenizer, device=None):
    # Tokenize the sequence with special tokens inserted, remove last token and replace with mask
    if type(prompt) is str:
        _input = tokenizer.encode(prompt, return_tensors="pt")
        _input[0, -1] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)
        _temp_tensor = torch.zeros((_input.shape[0], 1), dtype=torch.int64)
        _temp_tensor[0][0] = tokenizer.sep_token_id
        return torch.cat((_input, _temp_tensor), dim=1)
    else: # if it is tensor
        _input = prompt.clone()
        _temp_tensor = torch.zeros((_input.shape[0], 2), dtype=torch.int64)
        _temp_tensor[0][0] = tokenizer.mask_token_id
        _temp_tensor[0][1] = tokenizer.sep_token_id
        _temp_tensor = _temp_tensor.to(device)
        return torch.cat((prompt, _temp_tensor), dim=1)
_add_mask_to_last(prompt, tokenizer)

tensor([[   0, 1783,  275,  566,  278,  333,  275,  282,    4,    2]])

In [489]:
tokenizer.sep_token_id

2

In [490]:
device = torch.device("cuda:0")

In [537]:
cpu_device = torch.device("cpu")
model.to(cpu_device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(20000, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [491]:
def _predict_last_token(_input, tokenizer, k, top_p):
    """top_k – (optional) int The number of highest probability vocabulary tokens to keep for top-k-filtering. Between 1 and infinity. Default to 50.

    top_p – (optional) float The cumulative probability of parameter highest probability vocabulary tokens to keep for nucleus sampling. Must be between 0 and 1. Default to 1."""
    mask_token_index = torch.where(_input == tokenizer.mask_token_id)[1]
    _input.to(device)
    token_logits = model(_input)[0]
    mask_token_logits = token_logits[0, mask_token_index, :]
    _mask_token_logits_cpu = mask_token_logits.clone().cpu()
    top_k_tokens = torch.topk(_mask_token_logits_cpu, k, dim=1).indices[0].numpy()
    if np.random.random() < top_p:
        return top_k_tokens[0]
    else:
        return top_k_tokens[1:][np.random.randint(low=0, high=top_k_tokens.size-1)]

In [509]:
def _text_generate(prompt, tokenizer, length = 250, top_k=10, top_p=0.95, device=None):
#     _input = _add_mask_to_last(prompt, tokenizer)
    _input = prompt
    for i in range(length):
        _input = _add_mask_to_last(_input, tokenizer, device=device)
        _input = _input.to(device)
        _input[0, -2] = _predict_last_token(_input, tokenizer, k=top_k, top_p = top_p)
        _input = _input[:, :-1].clone()
    print(tokenizer.decode(_input[0]))

In [510]:
_input.to(device)

tensor([[   0, 1783,  275,  566,  278,  333,  275,  282,    4]],
       device='cuda:0')

In [528]:
_text_generate("""การตั้งกระทู้และการเชียร์บอล รวมถึงกิจกรรมทั้งหมดของชมรม ฯ เป็นไปเพื่อความบันเทิงของสมาชิกทุกๆท่าน""", tokenizer, device=device,length = 200,top_k=8, top_p=0.85)

<s>การตั้งกระทู้และการเชียร์บอล รวมถึงกิจกรรมทั้งหมดของชมรม ฯ เป็นไปเพื่อความบันเทิงของสมาชิกทุกๆท่าน
และการตั้งกระทู้และการให้ความเห็น
โดยการตั้งกระทู้และอิสรภาพ
โดยทั่วไปของชมรม ฯ เป็นการเพื่อความสะดวกบํารุงรักษาบรรยากาศ
โดยการตั้งกระทู้และการที่มีกระทันหัน
โดยการตั้งกระทืและการให้ความเห็น
โดยการตั้งกระทู้และการให้ความเห็น
โดยการตั้งกระทู่และการให้เป็นกระทู้และการให้ความเห
น
โดยการขอให้มีสาระ
หรือ ดูถึงความคิดเห็น
โดยการตั้งกระทู้และการตั้งกระทุ้
โดยการตั้งกระทู้และการให้ความเห็น
โดยการตั้งกระทู้ และการตั้งกระทู้
โดยการตั้งกระทู้และการให้เป็นกระทู้โดยการตั้งกระทู้และการที่มีกระทู้และการให้ความเห็น
โดยการตั้งกระทู้และการตั้งกระทู้และเพื่


In [532]:
_text_generate("""ตอนนี้เป็นเวลา""", tokenizer, device=device,length = 200,top_k=8, top_p=0.85)

<s>ตอนนี้เป็นเวลา 2 ปีแลด้วย และได้รับ sms ขยะที่มีข่าวออกมาว่าน่าจะมาด้วย และได้รับ dtac ขยะที่มีกระแสข่าวออกมาว่าน่าจะมาด้วย และไดเรรีย์ค่าย ais ขยะที่มั่นคงด้วย และกลายเป็น dtac ขยะที่มั่นคงไปด้วย และแพ็ก ais ขยะที่มั่นใจคงไปด้วย และยังมีข่าวออกมาวันนี้ท่ามกลางกระแสวิตกังวลอย่างมาก และกลายเป็น dtac ไดเรรีย์ค้าย ais ขยะ
ส่วนตัว เป็น dtac ขยะที่ต้องการขายออนไลน์ และกลายเป็น dtac ขยะที่มั่นคงไปด้วย และเป้าหมายของ dtac ขยะทุกคน เป็น dtac ขยะ โดยที่มี dtac ขยะทุกคนมี dtac ขยะทุกคน เป


In [308]:
top_k_tokens.size

10

In [306]:
np.random.random()

0.48881225040001264

In [43]:
prompt_length = len(tokenizer.decode(_inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
prompt_length

10

In [282]:
outputs = model.generate(_inputs, max_length=250, do_sample=True, top_p=0.95, top_k=5)
outputs

tensor([[1783,  275,  566,  278,  333,  275,  282,  282,  282,  282,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,  404,
          404,  404,  404,  

In [34]:
generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]
generated

'สวัสดีครับ                                                                                                                                                                                                                                                   '

In [46]:
tokenizer.decode(outputs[0])

'สวัสดีครับบบบบบbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb'